# Cart Pole - Policy Gradient Method
## Policy Gradients Vanilla type
* Borrowed heavily from structure of: [Vanilla Policy Gradient](https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb)
* In order to determine positive and negative rewards we must associate it with some direction of the pole

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim  # Easy model building, not quite sure yet how to build nodes by hand
import numpy as np
import matplotlib.pyplot as plt
import gym
%matplotlib inline

In [21]:
env = gym.make('CartPole-v0')
print(env.observation_space)  # 4x1 Box (vector-like)
print(env.action_space)       # 2x1 Discrete

[2017-04-17 14:34:25,147] Making new env: CartPole-v0


Box(4,)
Discrete(2)


In [1]:
class rl_agent():
    def __init__(self,  
                 environment, 
                 state_space_size, 
                 action_space_size,         # For CartPole-v0
                 history_size,
                 learning_rate,             # Lambda or other for gradient descent
                 epsilon,                   # Error - error for types of gradient descent or random choice
                 gradient_descent_function):
        tf.reset_default_graph()
        self.history_size = history_size
        self.state_space_size = state_space_size
        self.action_space_size = action_space_size
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.GDFunction = gradient_descent_function
        
        # TF Vars
        self.gradients = [] 
        self.state = tf.placeholder(shape=[,self.state_space_size], dtype=tf.float32)
        
        # ReLu is default
        self.hidden_layer = slim.fully_connected(self.state,
                                                 self.history_size,
                                                 biases_initializer=None)
        # Output with softmax (only two possible choices, but faster to do softmax)
        self.output_layer = slim.fully_connected(self.hidden_layer,
                                                 self.action_space_size,
                                                 biases_initializer=None,
                                                 activation_fn=tf.nn.softmax())
        self.current_action = tf.argmax(self.output_layer,1)
        self.reward_tensor = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_tensor = tf.placeholder(shape=[None],dtype=tf.int32)
        
        # Range from 0 to the output dimension -- an index range [ 0, 1, 2...
        output_range = tf.range(0, tf.shape(self.output_layer)[0])
        # Basically this outputs the action 
        self.indexes = output_range * tf.shape(self.output_layer)[1] + self.action_tensor
        # Formed an action tensor 
        self.output_tensor = tf.gather(tf.reshape(self.output_layer, [-1]), self.indexes)
        # Basically a spread and gather according to the index yielding output
        # which will be like y * y^ using the reduce mean here so we can reduce to 1x1
        self.loss_function = -tf.reduce_mean(tf.log(self.output_tensor)*self.reward_tensor)
        
        self.trainable_variables = tf.trainable_variables()
        
        self.gradient_holders = []
        # Each trainable variable is a partial derivative, but here they are just placeholders
        for idx,var in enumerate(self.trainable_variables):
            temp = tf.placeholder(tf.float32, name=str(idx)+'_holder')
            self.gradient_holders.append(temp)
        
        self.gradients = tf.gradients(self.loss_function, self.trainable_variables)
        
        optimizer = self.GDFunction(learning_rate=self.learning_rate)
        self.updated_weights = optimizer.apply_gradients(zip(self.gradient_holders,trainable_variables))
        
    def choose_action(self, session, s):
        return session.run(self.output_layer, feed_dict={self.state:[s]})

SyntaxError: invalid syntax (<ipython-input-1-9ed75403984c>, line 21)

In [2]:
agent = rl_agent(
    environment=env,
    state_space_size=4,
    action_space_size=2,
    learning_rate=0.001,
    epsilon=1e-8, 
    gradient_descent_function=tf.train.AdamOptimizer)

    init = 
        self.session.run(self.init)
        trainable_variables = tf.trainable_variables()
        self.gradients = self.session.run(trainable_variables)

IndentationError: unexpected indent (<ipython-input-2-96cdcad7ea2e>, line 10)